# Workshop DL01: Intro to Deep Learning

## Learning Objectives
* High-level understanding of the major components of deep neural networks - neurons, weights, biases, activation function
* Understanding of how to construct simple fully-connected deep neural networks using TensorFlow

## Worked Example: MNIST Digit Recognition

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

/Users/evan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Read the data

In [3]:
# read-in raw data
data = pd.read_csv("../datasets/mnist_train.csv")

# mask for splitting data into train and test sets
mask = np.random.rand(len(data)) < 0.90

# training data, features (X) and labels (y)
train_X = data[mask].drop("label", axis=1)
train_y = data.label[mask]

# test data, features (X) and labels (y)
test_X = data[~mask].drop("label", axis=1)
test_y = data.label[~mask]

In [19]:
# batchify
def batchify(data, batch_size):
    return np.array_split(data, len(data)//batch_size)

### Define the network

In [153]:
N_PIXELS = 28*28
N_CLASSES = 10

In [154]:
# (1) define the input placeholder tensor
X = tf.placeholder(tf.float32, shape=[None,N_PIXELS])

In [155]:
# (2) define the network architecture
h1 = tf.layers.dense(inputs=X, units=500, activation=tf.nn.relu)
h2 = tf.layers.dense(inputs=h1, units=500, activation=tf.nn.relu)

In [156]:
# (3) define the output layer
y_ = tf.layers.dense(inputs=h2, units=N_CLASSES, activation=None)

In [189]:
# (4) define the loss function
y = tf.placeholder(tf.int32, shape=[None])
y_one_hot = tf.one_hot(y, depth=N_CLASSES)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_, labels=y_one_hot))

In [190]:
# (5) define the optimizer
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [203]:
# (6) define accuracy statistic
correct = tf.equal(tf.argmax(y_one_hot, axis=1), tf.argmax(y_, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

### Train the network

In [204]:
N_TRAINING_EPOCHS = 10
TRAINING_BATCH_SIZE = 128

In [207]:
batches_X = batchify(train_X, TRAINING_BATCH_SIZE)
batches_y = batchify(train_y, TRAINING_BATCH_SIZE)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(N_TRAINING_EPOCHS):
        
        epoch_loss = 0
        for i in range(len(batches_X)):
            _, batch_loss = sess.run([optimizer, loss],
                feed_dict={X: batches_X[i], y: batches_y[i]})
            epoch_loss += batch_loss
            
        epoch_score = sess.run([accuracy],
                feed_dict={X: test_X, y: test_y})
        
        print("==================================")
        print("Epoch " + str(epoch) + " completed")
        print("Loss: " + str(epoch_loss))
        print("Acc:  " + str(epoch_score))
        print("==================================")

Epoch 0 completed
Loss: 1060.1766787171364
Acc:  [0.9255143]
Epoch 1 completed
Loss: 114.1358534488827
Acc:  [0.9406479]
Epoch 2 completed
Loss: 66.8469255939126
Acc:  [0.94726884]
Epoch 3 completed
Loss: 52.774382788455114
Acc:  [0.9512887]
Epoch 4 completed
Loss: 45.07347659629886
Acc:  [0.95790964]
Epoch 5 completed
Loss: 44.431201502652584
Acc:  [0.956018]
Epoch 6 completed
Loss: 47.255663134754286
Acc:  [0.9583826]
Epoch 7 completed
Loss: 45.367300333346066
Acc:  [0.9567274]
Epoch 8 completed
Loss: 42.524391990420554
Acc:  [0.9657129]
Epoch 9 completed
Loss: 32.241352115311656
Acc:  [0.96287537]
